# Projections and Operators Reference Guide

This guide demonstrates projections and operators in Trace - two core features for constrained optimization and flexible computation graphs.

## Overview

**Operators** are traced functions that enable computation on nodes while preserving the computation graph.

**Projections** are functions that constrain parameters to valid ranges or formats during optimization.

Let's explore each with practical examples.

In [1]:
# Import required modules
import numpy as np
from typing import Any

from opto.trace import node, bundle
from opto.trace.nodes import ParameterNode
from opto.trace.projections import Projection, BlackCodeFormatter
import opto.trace.operators as operators

print("Imports successful")

Imports successful


## Understanding Operators

Operators in Trace are traced functions that enable computation on nodes while preserving the computation graph. They support automatic differentiation for optimization.

In [2]:
# Basic operator usage
x = node(10)
y = node(5)

# Arithmetic operations
sum_result = operators.add(x, y)
product = operators.multiply(x, y) 
power_result = operators.power(x, node(2))

print(f"Addition: {x.data} + {y.data} = {sum_result.data}")
print(f"Multiplication: {x.data} * {y.data} = {product.data}")
print(f"Power: {x.data}^2 = {power_result.data}")

# String operations
text1 = node("Hello")
text2 = node(" World")
concatenated = operators.add(text1, text2)
uppercase = operators.upper(text1)

print(f"String concatenation: '{text1.data}' + '{text2.data}' = '{concatenated.data}'")
print(f"Uppercase: '{text1.data}' -> '{uppercase.data}'")

Addition: 10 + 5 = 15
Multiplication: 10 * 5 = 50
Power: 10^2 = 100
String concatenation: 'Hello' + ' World' = 'Hello World'
Uppercase: 'Hello' -> 'HELLO'


### Operator Categories

Trace provides operators across multiple categories:

In [3]:
# 1. Arithmetic Operators
a, b = node(15), node(4)

print("Arithmetic Operations:")
print(f"Addition: {operators.add(a, b).data}")
print(f"Subtraction: {operators.subtract(a, b).data}")
print(f"Multiplication: {operators.multiply(a, b).data}")
print(f"Division: {operators.divide(a, b).data}")
print(f"Floor Division: {operators.floor_divide(a, b).data}")
print(f"Modulo: {operators.mod(a, b).data}")
print(f"Power: {operators.power(a, node(2)).data}")

# 2. Mathematical Functions  
print("\nMathematical Functions:")
print(f"Floor: {operators.floor(node(3.7)).data}")
print(f"Ceiling: {operators.ceil(node(3.2)).data}")
print(f"Truncate: {operators.trunc(node(3.9)).data}")
print(f"Negative: {operators.neg(node(5)).data}")
print(f"Positive: {operators.pos(node(-5)).data}")

Arithmetic Operations:
Addition: 19
Subtraction: 11
Multiplication: 60
Division: 3.75
Floor Division: 3
Modulo: 3
Power: 225

Mathematical Functions:
Floor: 3
Ceiling: 4
Truncate: 3
Negative: -5
Positive: -5


In [4]:
# 3. Comparison and Logical Operations
x, y = node(10), node(5)

print("Comparison Operations:")
print(f"Less than: {x.data} < {y.data} = {operators.lt(x, y).data}")
print(f"Equal: {x.data} == {y.data} = {operators.eq(x, y).data}")
print(f"Greater than: {x.data} > {y.data} = {operators.gt(x, y).data}")

print("\nLogical Operations:")
condition = operators.gt(x, y)
true_val = node("x is greater")
false_val = node("x is smaller")
result = operators.cond(condition, true_val, false_val)
print(f"Conditional: {result.data}")

bool_val = node(True)
print(f"Logical NOT: not {bool_val.data} = {operators.not_(bool_val).data}")

Comparison Operations:
Less than: 10 < 5 = False
Equal: 10 == 5 = False
Greater than: 10 > 5 = True

Logical Operations:
Conditional: x is greater
Logical NOT: not True = False


In [5]:
# 4. Collection Operations
my_list = node([1, 2, 3, 4, 5])
my_dict = node({"a": 1, "b": 2, "c": 3})
my_string = node("Hello World")

print("Collection Operations:")
print(f"List length: {operators.len_(my_list).data}")
print(f"List indexing [2]: {operators.getitem(my_list, node(2)).data}")
print(f"Dict keys: {operators.keys(my_dict).data}")
print(f"Dict values: {operators.values(my_dict).data}")

# Membership testing
print(f"\n3 in list: {operators.in_(node(3), my_list).data}")
print(f"'Hello' in string: {operators.in_(node('Hello'), my_string).data}")

Collection Operations:
List length: 5
List indexing [2]: 3
Dict keys: ['a', 'b', 'c']
Dict values: [1, 2, 3]

3 in list: True
'Hello' in string: True


In [6]:
# 5. String Operations
text = node("  Hello World  ")

print("String Operations:")
print(f"Original: '{text.data}'")
print(f"Uppercase: '{operators.upper(text).data}'")
print(f"Lowercase: '{operators.lower(text).data}'")
print(f"Strip whitespace: '{operators.strip(text).data}'")

# Advanced string operations
sentence = node("apple,banana,cherry")
words = operators.split(sentence, node(","))
print(f"\nSplit: {words.data}")

# String formatting
template = node("Hello {name}, you are {age} years old")
formatted = operators.format(template, name="Alice", age=30)
print(f"Formatted string: {formatted.data}")

String Operations:
Original: '  Hello World  '
Uppercase: '  HELLO WORLD  '
Lowercase: '  hello world  '
Strip whitespace: 'Hello World'

Split: ['apple', 'banana', 'cherry']
Formatted string: Hello Alice, you are 30 years old


## Understanding Projections

Projections are functions that constrain parameters to valid ranges or formats during optimization. They ensure parameters remain within feasible sets.

In [7]:
# Basic projection example
class BoundedProjection(Projection):
    """Project values to a bounded range [min_val, max_val]."""
    
    def __init__(self, min_val: float, max_val: float):
        super().__init__()
        self.min_val = min_val
        self.max_val = max_val
    
    def project(self, x: Any) -> Any:
        """Clip value to the specified bounds."""
        if isinstance(x, (int, float)):
            return max(self.min_val, min(self.max_val, x))
        elif isinstance(x, np.ndarray):
            return np.clip(x, self.min_val, self.max_val)
        elif isinstance(x, list):
            return [max(self.min_val, min(self.max_val, val)) for val in x]
        return x

# Test the projection
projection = BoundedProjection(0.0, 1.0)

test_values = [-0.5, 0.3, 1.2, 0.8]
print("Bounded Projection (range [0.0, 1.0]):")
for val in test_values:
    projected = projection.project(val)
    print(f"  {val:5.1f} -> {projected:5.1f}")

Bounded Projection (range [0.0, 1.0]):
   -0.5 ->   0.0
    0.3 ->   0.3
    1.2 ->   1.0
    0.8 ->   0.8


In [8]:
# Using projections with ParameterNode
bounded_param = ParameterNode(
    0.5,
    name="learning_rate",
    description="Learning rate constrained to [0.001, 1.0]",
    projections=[BoundedProjection(0.001, 1.0)]
)

print(f"Initial parameter value: {bounded_param.data}")

# Simulate optimizer trying to set invalid values
test_updates = [10.0, -0.1, 0.5, 2.0]
print("\nProjecting invalid updates:")
for update in test_updates:
    # This is what happens internally during optimization
    projected = bounded_param.projections[0].project(update)
    print(f"  Update {update:5.1f} -> Projected {projected:6.3f}")

Initial parameter value: 0.5

Projecting invalid updates:
  Update  10.0 -> Projected  1.000
  Update  -0.1 -> Projected  0.001
  Update   0.5 -> Projected  0.500
  Update   2.0 -> Projected  1.000


### Custom Projections Examples

Here are practical examples of custom projections:

In [9]:
# Example 1: Probability Distribution Projection
class ProbabilityProjection(Projection):
    """Ensure values form a valid probability distribution."""
    
    def __init__(self, epsilon: float = 1e-8):
        super().__init__()
        self.epsilon = epsilon
    
    def project(self, x: Any) -> Any:
        """Normalize to valid probability distribution."""
        if isinstance(x, (list, np.ndarray)):
            x_array = np.array(x, dtype=float)
            # Ensure non-negative values
            x_array = np.maximum(x_array, self.epsilon)
            # Normalize to sum to 1
            x_array = x_array / np.sum(x_array)
            return x_array.tolist() if isinstance(x, list) else x_array
        return x

# Test probability projection
prob_proj = ProbabilityProjection()

test_distributions = [
    [0.1, 0.2, 0.3],      # Doesn't sum to 1
    [-0.1, 0.6, 0.5],     # Has negative values
    [0, 0, 0],            # All zeros
    [1, 2, 3],            # Arbitrary positive values
]

print("Probability Distribution Projection:")
print("Input -> Projected (Sum)")
for dist in test_distributions:
    projected = prob_proj.project(dist)
    print(f"{str(dist):20} -> {[round(p, 3) for p in projected]} (sum={sum(projected):.3f})")

Probability Distribution Projection:
Input -> Projected (Sum)
[0.1, 0.2, 0.3]      -> [0.167, 0.333, 0.5] (sum=1.000)
[-0.1, 0.6, 0.5]     -> [0.0, 0.545, 0.455] (sum=1.000)
[0, 0, 0]            -> [0.333, 0.333, 0.333] (sum=1.000)
[1, 2, 3]            -> [0.167, 0.333, 0.5] (sum=1.000)


In [10]:
# Example 2: Text Length Projection
class TextLengthProjection(Projection):
    """Constrain text to specific length range."""
    
    def __init__(self, min_length: int, max_length: int, pad_char: str = "."):
        super().__init__()
        self.min_length = min_length
        self.max_length = max_length
        self.pad_char = pad_char
    
    def project(self, x: Any) -> Any:
        """Adjust text length to fit constraints."""
        if isinstance(x, str):
            if len(x) > self.max_length:
                return x[:self.max_length-3] + "..."
            elif len(x) < self.min_length:
                return x + self.pad_char * (self.min_length - len(x))
            return x
        return x

# Test text length projection
text_proj = TextLengthProjection(min_length=10, max_length=20)

test_texts = [
    "Hi",                                    # Too short
    "Perfect length",                       # Just right
    "This text is way too long for the constraint"  # Too long
]

print("Text Length Projection (10-20 chars):")
for text in test_texts:
    projected = text_proj.project(text)
    print(f"Input ({len(text):2d}): '{text}'")
    print(f"Output({len(projected):2d}): '{projected}'")
    print()

Text Length Projection (10-20 chars):
Input ( 2): 'Hi'
Output(10): 'Hi........'

Input (14): 'Perfect length'
Output(14): 'Perfect length'

Input (44): 'This text is way too long for the constraint'
Output(20): 'This text is way ...'



In [11]:
# Example 3: Composite Projections
class CompositeProjection(Projection):
    """Apply multiple projections in sequence."""
    
    def __init__(self, *projections: Projection):
        super().__init__()
        self.projections = projections
    
    def project(self, x: Any) -> Any:
        """Apply projections sequentially."""
        result = x
        for projection in self.projections:
            result = projection.project(result)
        return result

# Example: Combine bounds with probability normalization
bounded_prob_proj = CompositeProjection(
    BoundedProjection(0.0, 10.0),  # First ensure non-negative
    ProbabilityProjection()         # Then normalize
)

test_values = [[-1, 2, 3], [0.1, 0.2, 0.3], [5, 10, 15]]
print("Composite Projection (Bounds + Probability):")
for val in test_values:
    projected = bounded_prob_proj.project(val)
    print(f"Input:  {val}")
    print(f"Output: {[round(p, 3) for p in projected]}")
    print(f"Sum:    {sum(projected):.3f}")
    print()

Composite Projection (Bounds + Probability):
Input:  [-1, 2, 3]
Output: [0.0, 0.4, 0.6]
Sum:    1.000

Input:  [0.1, 0.2, 0.3]
Output: [0.167, 0.333, 0.5]
Sum:    1.000

Input:  [5, 10, 15]
Output: [0.2, 0.4, 0.4]
Sum:    1.000



## Built-in Projections

Trace provides built-in projections for common use cases:

In [12]:
# BlackCodeFormatter projection
try:
    code_projection = BlackCodeFormatter()
    
    sample_code = "def add(a,b): return a+b"
    
    print("BlackCodeFormatter Projection:")
    print(f"Input code:  {sample_code}")
    formatted_code = code_projection.project(sample_code)
    print(f"Formatted:\n{formatted_code}")
    
except ImportError:
    print("BlackCodeFormatter requires 'black' package")
    print("Install with: pip install black")
    print("\nBlackCodeFormatter automatically formats Python code")
    print("It only processes strings containing 'def' keyword")

BlackCodeFormatter Projection:
Input code:  def add(a,b): return a+b
Formatted:
def add(a, b):
    return a + b



## Integration with Optimization

Projections work seamlessly with Trace optimizers:

In [13]:
# Example: Parameter with projection
from opto.optimizers import OptoPrime

# Create a parameter with bounds projection
constrained_param = ParameterNode(
    0.5,
    name="weight",
    description="Weight parameter bounded to [0, 1]",
    projections=[BoundedProjection(0.0, 1.0)]
)

print("Parameter with Projection:")
print(f"Initial value: {constrained_param.data}")
print(f"Has projections: {len(constrained_param.projections) > 0}")
print(f"Projection type: {type(constrained_param.projections[0]).__name__}")

# Create a simple optimization problem
@bundle()
def objective_function(weight):
    """Simple quadratic objective."""
    # Minimize (weight - 0.3)^2
    return (weight - 0.3) ** 2

# Test projection effect
result = objective_function(constrained_param)
print(f"\nObjective value: {result.data:.4f}")

# Simulate what happens during optimization
print("\nSimulating optimization updates:")
simulated_updates = [1.5, -0.2, 0.8, 2.0]
for update in simulated_updates:
    projected = constrained_param.projections[0].project(update)
    print(f"Update {update:4.1f} -> Projected {projected:4.1f}")

Parameter with Projection:
Initial value: 0.5
Has projections: True
Projection type: BoundedProjection

Objective value: 0.0400

Simulating optimization updates:
Update  1.5 -> Projected  1.0
Update -0.2 -> Projected  0.0
Update  0.8 -> Projected  0.8
Update  2.0 -> Projected  1.0


/home/aswaminathan/miniconda3/envs/trace/lib/python3.9/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


## Advanced Patterns

### Dynamic Projection Selection

In [14]:
# Conditional projection based on value
class ConditionalProjection(Projection):
    """Apply different projections based on conditions."""
    
    def __init__(self, condition_func, true_projection, false_projection):
        super().__init__()
        self.condition_func = condition_func
        self.true_projection = true_projection
        self.false_projection = false_projection
    
    def project(self, x: Any) -> Any:
        """Apply projection based on condition."""
        if self.condition_func(x):
            return self.true_projection.project(x)
        else:
            return self.false_projection.project(x)

# Example: Different bounds for positive vs negative values
def is_positive(x):
    return isinstance(x, (int, float)) and x > 0

conditional_proj = ConditionalProjection(
    condition_func=is_positive,
    true_projection=BoundedProjection(0.1, 10.0),   # Positive values
    false_projection=BoundedProjection(-10.0, -0.1) # Negative values
)

test_values = [-15, -0.05, 0.05, 15]
print("Conditional Projection (different bounds for +/- values):")
for val in test_values:
    projected = conditional_proj.project(val)
    condition = "positive" if is_positive(val) else "negative"
    print(f"{val:5.2f} ({condition:8}) -> {projected:6.2f}")

Conditional Projection (different bounds for +/- values):
-15.00 (negative) -> -10.00
-0.05 (negative) ->  -0.10
 0.05 (positive) ->   0.10
15.00 (positive) ->  10.00


### Operator Chaining with Complex Data

In [15]:
# Complex computation using operators
def process_data(data_dict):
    """Process dictionary data using operators."""
    # Extract values using operators
    keys = operators.keys(data_dict)
    values = operators.values(data_dict)
    
    # Compute statistics
    total = node(0)
    count = operators.len_(values)
    
    # Sum all values
    values_list = values.data
    for val in values_list:
        total = operators.add(total, node(val))
    
    average = operators.divide(total, count)
    
    # Create result dictionary
    result = {
        "total": total.data,
        "count": count.data,
        "average": average.data,
        "summary": f"Total: {total.data}, Count: {count.data}, Average: {average.data:.2f}"
    }
    
    return result

# Test complex computation
test_data = node({"a": 10, "b": 20, "c": 30, "d": 40})
result = process_data(test_data)

print("Complex Computation with Operators:")
print(f"Input: {test_data.data}")
print(f"Total: {result['total']}")
print(f"Count: {result['count']}")
print(f"Average: {result['average']}")
print(f"Summary: {result['summary']}")

Complex Computation with Operators:
Input: {'a': 10, 'b': 20, 'c': 30, 'd': 40}
Total: 100
Count: 4
Average: 25.0
Summary: Total: 100, Count: 4, Average: 25.00


## Summary

This guide covered the essential features of projections and operators in Trace:

### Operators
- Enable traced computation while preserving differentiability
- Cover arithmetic, logical, comparison, and data manipulation operations
- Support diverse data types and maintain computation graphs
- Essential for building complex, optimizable functions

### Projections
- Enforce constraints on parameters during optimization
- Are automatically applied by optimizers
- Can be composed for complex constraint scenarios
- Enable constrained optimization in diverse domains

### Key Concepts
- **Operators**: Use for all computations that need to be traced
- **Projections**: Design to be idempotent and efficient
- **Integration**: Both work seamlessly with Trace optimizers
- **Composition**: Multiple projections can be chained together

### Implementation Notes
- All operations are captured in the computation graph
- Projections are applied during parameter updates
- Error handling is built into the tracing system
- Support for parallel execution where applicable

These features provide the foundation for building sophisticated, constrained optimization systems using Trace.